In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('transdf.csv')
df.shape

(800, 79)

In [2]:
target = 'class'
preds = [x for x in df.columns if x not in [target]]
X_train = df[preds]
y_train = df[target]

In [3]:
from featurewiz import FeatureWiz

Imported lazytransform v1.18. 

Imported featurewiz 0.6.1. Use the following syntax:
    >>> wiz = FeatureWiz(feature_engg = '', nrows=None, transform_target=True,
        		category_encoders="auto", auto_encoders='VAE', ae_options={},
        		add_missing=False, imbalanced=False, verbose=0)
    >>> X_train_selected, y_train = wiz.fit_transform(X_train, y_train)
    >>> X_test_selected = wiz.transform(X_test)
    >>> selected_features = wiz.features
    


In [4]:
wiz = FeatureWiz(feature_engg = '', nrows=None, transform_target=False,
        		category_encoders="auto", auto_encoders='', ae_options={},
        		add_missing=False, imbalanced=False, verbose=0)
X_trans, y_trans = wiz.fit_transform(X_train, y_train)
print('Final selected: ', wiz.features)

featurewiz is given 0.9 as correlation limit...
    Skipping feature engineering since no feature_engg input...
    final list of category encoders given: ['onehot', 'label']
    final list of scalers given: []
Loaded input data. Shape = (800, 78)
#### Starting featurewiz transform for train data ####
    Single_Label Binary_Classification problem 
Shape of dataset: (800, 78). Now we classify variables into different types...
Time taken to define data pipeline = 1 second(s)
No model input given...
Lazy Transformer Pipeline created...
    Time taken to fit dataset = 1 second(s)
    Time taken to transform dataset = 1 second(s)
    Shape of transformed dataset: (800, 119)
    Single_Label Binary_Classification problem 
Starting SULOV with 119 features...
    there are no null values in dataset...
    there are no null values in target column...
Completed SULOV. 91 features selected
Performing recursive XGBoost feature selection from 91 features...
    time taken to run entire featurewiz 

In [5]:
testdf = pd.read_csv('testdf.csv')
X_test = testdf[preds]
y_test = testdf[target]

In [6]:
X_test_trans = wiz.transform(X_test)

#### Starting featurewiz transform for test data ####
Loaded input data. Shape = (200, 78)
#### Starting lazytransform for test data ####
    Time taken to transform dataset = 1 second(s)
    Shape of transformed dataset: (200, 119)
Returning dataframe with 34 features 


In [7]:
from catboost import CatBoostClassifier, CatBoostRegressor
cats = X_train.select_dtypes(include=['bool', 'object', 'category']).columns.to_list()
base = CatBoostClassifier(cat_features=cats, verbose=0)

In [8]:
### train a model on original features
base.fit(X_train, y_train)
y_pred = base.predict(X_test)

In [9]:
from print_metrics import print_classification_metrics, print_regression_metrics
print_classification_metrics(y_test, y_pred, verbose=1)

Bal accu 56%
              precision    recall  f1-score   support

           0       0.73      0.98      0.84       141
           1       0.73      0.14      0.23        59

    accuracy                           0.73       200
   macro avg       0.73      0.56      0.53       200
weighted avg       0.73      0.73      0.66       200

final average balanced accuracy score = 0.56


0.5571583122971511

In [10]:
from catboost import CatBoostClassifier, CatBoostRegressor
cats = X_trans.select_dtypes(include=['bool', 'object', 'category']).columns.to_list()
eng = CatBoostClassifier(cat_features=cats, verbose=0)

In [11]:
eng.fit(X_trans, y_trans)
y_pred_trans = eng.predict(X_test_trans)

In [12]:
from print_metrics import print_classification_metrics, print_regression_metrics
print_classification_metrics(y_test, y_pred_trans, verbose=1)

Bal accu 50%
              precision    recall  f1-score   support

           0       0.70      1.00      0.83       141
           1       0.00      0.00      0.00        59

    accuracy                           0.70       200
   macro avg       0.35      0.50      0.41       200
weighted avg       0.50      0.70      0.58       200

final average balanced accuracy score = 0.50


0.5

In [13]:
from featurewiz_polars import Featurewiz_MRMR, Polars_CategoricalEncoder
import polars as pl
import pyarrow

Imported featurewiz_polars 0.1.6. Use the following syntax:
 >> from featurewiz_polars import Featurewiz_MRMR, Featurewiz_MRMR_Model
    


In [14]:
mrmr = Featurewiz_MRMR(model_type='classification', encoding_type='target', classic=True, verbose=1)
Xt, yt = mrmr.fit_transform(pl.DataFrame(X_train), pl.Series(y_train))
Xt_trans = mrmr.transform(pl.DataFrame(X_test))


Featurewiz Polars started. Model type: Classification

Polars Featurewiz SULOV-MRMR Feature Selection Algorithm
Initial Features: 78

Mutual Information Scores:
    checking_status: 0.0608
    autofe_(duration-age): 0.0556
    autofe_(duration/existing_credits): 0.0476
    autofe_(duration+existing_credits): 0.0420
    other_payment_plans: 0.0384
    autofe_(duration+installment_commitment): 0.0359
    autofe_(duration+residence_since): 0.0355
    autofe_(age/num_dependents): 0.0353
    autofe_(duration+num_dependents): 0.0350
    autofe_(duration*residence_since): 0.0333
    autofe_(duration-existing_credits): 0.0322
    autofe_(duration-residence_since): 0.0317
    autofe_(duration/num_dependents): 0.0307
    purpose: 0.0273
    autofe_(age-installment_commitment): 0.0266
    autofe_(duration*existing_credits): 0.0264
    personal_status: 0.0260
    autofe_(credit_amount*installment_commitment): 0.0259
    property_magnitude: 0.0256
    autofe_(age+num_dependents): 0.0254
    autofe

In [15]:
from catboost import CatBoostClassifier, CatBoostRegressor
cats = Xt.to_pandas().select_dtypes(include=['bool', 'object', 'category']).columns.to_list()
pol = CatBoostClassifier(cat_features=cats, verbose=0)

In [16]:
Xt.head(1)

autofe_(duration+num_dependents),checking_status,credit_history,num_dependents,other_parties,savings_status,property_magnitude,installment_commitment,own_telephone,personal_status,autofe_(duration-age),autofe_(duration-credit_amount),autofe_(duration/credit_amount),autofe_(credit_amount*installment_commitment),autofe_(duration+installment_commitment)
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
61.0,0.490654,0.30137,1.0,0.304167,0.350211,0.439024,3.0,0.285714,0.261364,-3.0,-6776.0,0.008777,20508.0,63.0


In [17]:
Xt_trans.head(1)

autofe_(duration+num_dependents),checking_status,credit_history,num_dependents,other_parties,savings_status,property_magnitude,installment_commitment,own_telephone,personal_status,autofe_(duration-age),autofe_(duration-credit_amount),autofe_(duration/credit_amount),autofe_(credit_amount*installment_commitment),autofe_(duration+installment_commitment)
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
22.23875,0.490654,0.321429,1.0,0.304167,0.350211,0.2,2.0,0.3125,0.362205,-14.46375,-3339.52375,0.008729,9035.95,24.06


In [18]:
pol.fit(Xt.to_pandas(), yt.to_pandas())
y_pred_pol = pol.predict(Xt_trans.to_pandas())

In [19]:
from print_metrics import print_classification_metrics, print_regression_metrics
print_classification_metrics(y_test, y_pred_pol, verbose=1)

Bal accu 59%
              precision    recall  f1-score   support

           0       0.74      0.95      0.83       141
           1       0.65      0.22      0.33        59

    accuracy                           0.73       200
   macro avg       0.70      0.59      0.58       200
weighted avg       0.72      0.73      0.69       200

final average balanced accuracy score = 0.59


0.5853467964899628